In [11]:
from copy import copy
from time import time

In [2]:
filename = 'lexicon.txt'
lexicon_file = open(filename, 'r')
lexicon = [line.rstrip() for line in lexicon_file.readlines()]

In [43]:
class TrieNode:
    def __init__(self, eow: bool = False):
        self.eow = eow  # end of word
        self.children = {}

    def add(self, char, final_char: bool = False):
        assert char not in self.children.keys()
        self.children[char] = TrieNode(final_char)

    def __repr__(self):
        return '{} {}'.format('eow' if self.eow else '', self.children, )


class Trie:
    def __init__(self):
        self.root = TrieNode()

    def add_word(self, word: str):
        if word in self:
            return

        word = list(word)
        node = self.root
        while word[0] in node.children.keys():
            node = node.children[word.pop(0)]
        while word:
            char = word.pop(0)
            node.add(char, True if not word else False)
            node = node.children[char]

    def add_words(self, words: list):
        for word in words:
            self.add_word(word)

    def find_words(self, letters: list):
        valid_words = set()
        search_candidate = []
        letters_remaining = copy(letters)

        def search(v):
            if v.eow and ''.join(search_candidate) not in valid_words:
                valid_words.add(''.join(search_candidate))

            for l in set(letters_remaining):
                if l in v.children:
                    search_candidate.append(l)
                    letters_remaining.remove(l)
                    search(v.children[l])
                    del search_candidate[-1]
                    letters_remaining.append(l)

        search(self.root)
        return valid_words

    def get_lexicon(self):
        lexicon = set()
        search_candidate = []

        def search(v):
            if v.eow and ''.join(search_candidate) not in lexicon:
                lexicon.add(''.join(search_candidate))

            for l, w in v.children.items():
                search_candidate.append(l)
                search(w)
                del search_candidate[-1]

        search(self.root)
        return lexicon

    def __contains__(self, word: str):
        word = list(word)
        node = self.root
        while word:
            char = word.pop(0)
            try:
                node = node.children[char]
            except KeyError:
                return False
        return True if node.eow else False

    def __repr__(self):
        return str(self.root)



In [44]:
trie = Trie()
trie.add_words(lexicon)

In [48]:
ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
           'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'A', 'A', 'E', 'E']
start = time()
words = trie.find_words(ALPHABET*3)
time() - start

2.0123534202575684

In [46]:
len(words)

59028

In [21]:
start = time()
trie.get_lexicon()
time() - start

0.5148344039916992